In [1]:
sc

In [6]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

## Summary Statistics

In [8]:
from collections import namedtuple
Record = namedtuple('Record',['desc','value1','value2'])

In [9]:
recDF = sc.parallelize([Record("first",1,3.7),
                        Record("second",-2,2.1),
                        Record("third",6,0.7)]).toDF()

In [11]:
recStats = recDF.describe()
recStats.show()

+-------+------------------+------------------+
|summary|            value1|            value2|
+-------+------------------+------------------+
|  count|                 3|                 3|
|   mean|1.6666666666666667| 2.166666666666667|
| stddev| 4.041451884327381|1.5011106998930273|
|    min|                -2|               0.7|
|    max|                 6|               3.7|
+-------+------------------+------------------+



In [12]:
recStatsPandas = recStats.toPandas().set_index('summary')
recStatsPandas

,value1,value2
summary,,
count,3,3
mean,1.6666666666666667,2.166666666666667
stddev,4.041451884327381,1.5011106998930273
min,-2,0.7
max,6,3.7


In [16]:
recStatsPandas.loc['mean'].value1

u'1.6666666666666667'

In [13]:
recDF.stat.corr('value1','value2')

-0.5879120879120878

In [14]:
recDF.stat.cov('value1','value2')

-3.566666666666667

In [15]:
recDF.stat.freqItems(['value1','value2']).toPandas()

,value1_freqItems,value2_freqItems
0,"[-2, 1, 6]","[0.7, 2.1, 3.7]"


## Sampling

In [17]:
df = sqlc.createDataFrame([(1, 10), (1, 20), (2, 10),(2, 20), (2, 30), (3, 20), (3, 30)]).toDF("key", "value")

In [18]:
df.show()

+---+-----+
|key|value|
+---+-----+
|  1|   10|
|  1|   20|
|  2|   10|
|  2|   20|
|  2|   30|
|  3|   20|
|  3|   30|
+---+-----+



In [20]:
dfSampled = df.sample(withReplacement=False, fraction=0.3, seed=11)
dfSampled.show()

+---+-----+
|key|value|
+---+-----+
|  1|   10|
|  2|   10|
|  2|   30|
+---+-----+



In [21]:
training, testing = df.randomSplit(weights=[0.3, 0.7], seed=11)

In [22]:
training.show()

+---+-----+
|key|value|
+---+-----+
|  1|   10|
|  2|   10|
|  2|   30|
+---+-----+



In [23]:
testing.show()

+---+-----+
|key|value|
+---+-----+
|  1|   20|
|  2|   20|
|  3|   20|
|  3|   30|
+---+-----+



## Stratified Sampling

In [24]:
dfStrat = df.stat.sampleBy(col="key", fractions={1: 0.7, 2: 0.7, 3: 0.7}, seed=11)
dfStrat.show()

+---+-----+
|key|value|
+---+-----+
|  1|   10|
|  1|   20|
|  2|   10|
|  2|   30|
|  3|   20|
|  3|   30|
+---+-----+



## Random Data Generation

In [25]:
from pyspark.sql.functions import rand, randn

In [26]:
df = sqlc.range(0, 5)

In [27]:
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [28]:
df2 = df.select("id").withColumn("uniform", rand(5)).withColumn("normal", randn(5))

In [29]:
df2.show()

+---+-------------------+--------------------+
| id|            uniform|              normal|
+---+-------------------+--------------------+
|  0|0.47611851579756026|-0.21311682946326227|
|  1|0.06498948189958098|-0.05248092572410684|
|  2| 0.7069655052310547|  1.3682472758997855|
|  3| 0.1982919638208397|  -0.256535324205377|
|  4|0.12030715258495939|  -0.506853671746243|
+---+-------------------+--------------------+

